In [9]:
import re
import os
import string
import pandas as pd
import pandas_profiling
import numpy as np
import json
from pandas.io.json import json_normalize

pd.options.display.max_rows = 100
pd.options.display.max_columns = 70

# Tipos de campos a desconsiderar
# typeNotConsider = ['date','datetime','string','text']
# Tabela para processar
table = "maint_order"
# maint_req , maint_order

#============================================

dir_csv = "./base_manusis/original/"
data = pd.read_csv(dir_csv+table+".csv",low_memory=False)

# Pesos tipo do campo topsis
with open('./config/tipocampopeso.json') as json_file:  
    data_topsis = json.load(json_file)

def split_custom(str_p,delimiter_p):
    '''
    str_p = String to split
    delimiter_p = Caracter to split
    return the first string before delimiter
    '''
    for index,i in enumerate(str_p):
        if any(del_aux in i for del_aux in delimiter_p):
            return str_p[:index]
            break
    return str_p

In [10]:
f = open(dir_csv+table+".txt", "r",encoding='utf-8')
tt = f.read()
tt = tt.replace("\t", " ")
tt = tt.replace("\n", " ")
tt = tt.split('{')

k = []
for i in tt:
    y = []
    for t in i.split(','):
        
        u = t.strip()
        if any(c in u for c in ['desc','name','type','useNull']):
            try:
                u1 = u.split(':')
                u2 = u1[1].strip()
                u3 = np.NaN
                if any(c in u1 for c in ['desc','name','type']):
                    if len(re.findall(r"(?<=\')(.*?)(?=\')",u2)) > 0:
                        u3 = re.findall(r"(?<=\')(.*?)(?=\')",u2)
                        u3 = u3[0]

                    if len(re.findall(r'(?<=\")(.*?)(?=\")',u2)) > 0:
                        u3 = re.findall(r'(?<=\")(.*?)(?=\")',u2)
                        u3 = u3[0]

                if u1[0] == 'useNull':
                    u3 = u1[1]
                    u3 = u3.strip()
                    u3 = split_custom(u3,[' ',','])
                else:
                    pass
                y.append(u3)
            except:
                  pass  
    if len(y) == 4:
        k.append(y)
    else:
        pass

In [11]:
data_extrac = pd.DataFrame(k)
data_extrac = data_extrac.rename(columns={data_extrac.columns[0]: 'desc',
                                          data_extrac.columns[1]: 'name',
                                          data_extrac.columns[2]: 'type',
                                          data_extrac.columns[3]: 'usenull'})
data_extrac = data_extrac.dropna()
data_describe = data_extrac[['desc','type']].groupby(['type']).count()

# Percentual de registros nulos
data_missing = pd.DataFrame(data.isna().mean()*100)
data_missing.reset_index(level=0, inplace=True)
data_missing = data_missing.rename(columns={data_missing.columns[0]: 'name',data_missing.columns[1]:'percentnull'})

# Percentual de registros unicos
data_unique = pd.DataFrame(data.nunique())
data_unique.reset_index(level=0, inplace=True)
data_unique = data_unique.rename(columns={data_unique.columns[0]: 'name',data_unique.columns[1]:'percentunique'})
rows_data,cols_data = data.shape
data_unique['percentunique'] = (data_unique['percentunique']/rows_data)*100

data_analysis = pd.DataFrame(list(data.columns))
data_analysis = data_analysis.rename(columns={data_analysis.columns[0]: 'name'})

data_analysis = data_analysis.merge(data_extrac[['name','type','usenull']],how='left',on='name')
data_analysis = data_analysis.merge(data_missing,how='left',on='name')
data_analysis = data_analysis.merge(data_unique,how='left',on='name')
data_analysis['type'].fillna('other', inplace=True)

# Campo requerido 1 true 0 false
data_analysis['usenull'] = np.where(data_analysis['usenull']=='true', 1, 0)

# Pesos tipo de campos Json 
typevalues = json_normalize(data_topsis['type']).T
typevalues.reset_index(level=0, inplace=True)
typevalues = typevalues.rename(columns={typevalues.columns[0]: 'type',typevalues.columns[1]:'typevalue'})
typevalues = typevalues.sort_values(by='typevalue')

data_analysis = data_analysis.merge(typevalues,how='left',on='type')
data_analysis = data_analysis[['name','typevalue','usenull','percentnull','percentunique']]

In [12]:
# list(set(list(data.columns)) - set(data_extrac['name'].tolist()))

In [13]:
# data_analysis.type.drop_duplicates()

In [14]:
data_analysis.to_csv("./base_manusis/topsis"+table+".csv", index=False)

In [15]:
data_analysis

,name,typevalue,usenull,percentnull,percentunique
0,id,1,1,0.000000,100.000000
1,order_number,4,1,0.000000,86.331496
2,company_id,1,1,0.000000,0.000499
3,area_id,1,1,0.000000,0.003496
4,first_loc_id,1,1,1.580477,0.031460
5,second_loc_id,1,1,1.585221,0.156799
6,asset_id,1,1,1.340784,3.072317
7,asset_part_id,1,1,94.439617,1.203460
8,asset_part_child_id,1,1,99.085670,0.282638
9,asset_group_id,1,1,1.340784,0.379015
